In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from xgboost import XGBRegressor

import multiprocessing as mp
from functools import partial
from helper_functions import *

## Load Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv').sort_values(by=['cfips', 'first_day_of_month']).reset_index().drop('index', axis=1)
results_data = pd.read_csv('all_results_with_category.csv')

In [3]:
test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1001_2022-12-01,1001,2022-12-01
2,1001_2023-01-01,1001,2023-01-01
3,1001_2023-02-01,1001,2023-02-01
4,1001_2023-03-01,1001,2023-03-01
...,...,...,...
25075,56045_2023-02-01,56045,2023-02-01
25076,56045_2023-03-01,56045,2023-03-01
25077,56045_2023-04-01,56045,2023-04-01
25078,56045_2023-05-01,56045,2023-05-01


In [4]:
results_data.head()

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,XGBR_MAPE,tf_dense_MAPE,dense_W9_H8_MV,category
0,1001,0.732748,1.835232,0.992271,2.257356,1.504614,0.724775,Dense_MV
1,1003,0.747286,5.230218,4.317589,93.943110,4.143341,2.480568,LSTM
2,1005,1.273616,1.101416,1.833943,0.912357,1.395184,4.606141,XGBR
3,1007,9.315642,3.574082,4.259990,4.514549,5.228748,24.276476,SV
4,1009,0.766072,2.116541,1.285277,2.398140,1.442244,1.445454,LSTM


In [7]:
all_results = results_data[['Country', 'LSTM_W9_H8', 'SV_MAPE_x', 'LR_MAPE', 'dense_W9_H8_MV']]

best_score = all_results.min(axis='columns')

cat = []
for i, m in enumerate(best_score):
    if m == all_results['LSTM_W9_H8'].iloc[i]:
        category = 'LSTM'
    elif m == all_results['SV_MAPE_x'].iloc[i]:
        category = 'SV'
    elif m == all_results['LR_MAPE'].iloc[i]:
        category = 'LR'
    elif m == all_results['dense_W9_H8_MV'].iloc[i]:
        category = 'Dense_MV'
    cat.append(category)
all_results['category'] = cat

C:\Users\Masa\AppData\Local\Temp\ipykernel_17628\2445191668.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_results['category'] = cat


In [8]:
all_results

,Country,LSTM_W9_H8,SV_MAPE_x,LR_MAPE,dense_W9_H8_MV,category
0,1001,0.732748,1.835232,0.992271,0.724775,Dense_MV
1,1003,0.747286,5.230218,4.317589,2.480568,LSTM
2,1005,1.273616,1.101416,1.833943,4.606141,SV
3,1007,9.315642,3.574082,4.259990,24.276476,SV
4,1009,0.766072,2.116541,1.285277,1.445454,LSTM
...,...,...,...,...,...,...
3130,56037,2.913013,2.450784,3.550418,9.501690,SV
3131,56039,0.354971,1.586990,2.390681,3.563838,LSTM
3132,56041,4.173564,1.381605,3.925458,9.976927,SV
3133,56043,0.819441,3.608545,2.219479,6.841048,LSTM


In [9]:
%%time
if __name__ == '__main__':
    HORIZON = 8
    WINDOW_SIZE = 9

    train = pd.read_csv('train.csv')
    IDS = train.cfips.unique()
    
    sample_ids = IDS[:50]
    
    window_size = WINDOW_SIZE
    horizon = HORIZON
    epoch = 500
    split_size = 0
    
    cfips_list = []
    forecasts = []

    pool = mp.Pool(16)

    func = partial(get_score_for_competition, train, all_results, window_size, horizon, epoch, split_size)
    results = pool.map(func, IDS)
    pool.close()
    pool.join()
    
    for i in range(len(results)):
        c = results[i][0]
        cfips_list.append(c)
        forecasts.append(results[i][1])

CPU times: total: 6.55 s
Wall time: 54min 57s


In [10]:
test['microbusiness_density'] = np.array(forecasts).reshape((-1))
submission = test[['row_id', 'microbusiness_density']]

In [11]:
submission.head(10)

,row_id,microbusiness_density
0,1001_2022-11-01,3.169709
1,1001_2022-12-01,3.198731
2,1001_2023-01-01,3.213416
3,1001_2023-02-01,3.214808
4,1001_2023-03-01,3.208710
5,1001_2023-04-01,3.219334
6,1001_2023-05-01,3.234708
7,1001_2023-06-01,3.239502
8,1003_2022-11-01,8.237763
9,1003_2022-12-01,8.221723


In [12]:
submission.to_csv('submission_5.csv', index=False)